# A full business solution

## BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
will be provided a company name and their primary website.


In [14]:
# If these fail, check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [6]:
# Initialize and constants OPEN AI İLE
"""
Ne Yapar?
load_dotenv() fonksiyonu, .env dosyasındaki tüm çevresel değişkenleri yükler ve bu değişkenlere Python uygulamanızda erişmenize olanak tanır.
override=True Parametresinin Önemi:
Varsayılan olarak override=False olan load_dotenv() fonksiyonu, yalnızca ortamda (environment) mevcut olmayan değişkenleri yükler.
Ancak override=True kullanırsanız, mevcut çevresel değişkenlerin üzerine yazar. 
Bu, .env dosyanızdaki yeni değerlerin, sistemde ya da uygulamanızda daha önce belirlenmiş değerlerin yerine geçmesini sağlar.
"""
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [22]:
################OLLAMA İLE

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [23]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt
    
def get_links(url):
    website = Website(url)
    ####OPEN Aİ İLE
    """" 
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    """
    ## OLLAMA İLE YAPARSAN ollama_via_openai.chat.completions.create() KOY OPENAİ İLE YAPARSAN openai.chat.completions.create(
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Company page', 'url': 'https://status.huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Company/About page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter account', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


Hugging Face Brochure
======================

### Mission

At Hugging Face, we believe that the power of artificial intelligence can be harnessed by everyone. We're building an open source platform where the machine learning community collaborates, discovers, and deploys AI models on real-world problems.

### What We Do

Our platform is a central hub for:

*   **Training AI Models**: With over 1 million pre-trained models already available, our users can explore various applications such as natural language processing, computer vision, and audio analysis.
*   **Collaboration**: Host, discover, and reuse public models, datasets, and applications. Move faster with our open-source technology stack.
*   **Innovation**: Explore modalities like text, image, video, or audio. Build and share your skills on the platform.

### Our Impact

More than 50,000 organizations are using Hugging Face to accelerate their AI development. Collaborate with partners from diverse backgrounds including companies like Meta, Amazon Web Services, Google, Intel, Microsoft, Grammarly, Writer, etc. Join us in building a better future for machine learning by contributing your expertise.

### Our Open Source

We're committed to open source best practice and contribute to various libraries and repositories related to transformer models, natural language processing, safety critical AI components and more.

Hugging Face - Empowering AI Innovation
---------------------------------------

Be part of our community and let us work together to explore the possibilities of artificial intelligence for everyone.

[Get Started](https://huggingface.co)

[GitHub Repository](https://github.com/huggingface/transformers)

[Documentation](https://huggingface.co/docs)